In [2]:
import requests
import pandas as pd
import json
import urllib
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
import time
import random
import logging
import sys, traceback

In [109]:
logging.basicConfig(level=logging.ERROR)

In [3]:
driver = webdriver.Chrome('/Users/koojaehong/Jupyter_notebook/chromedriver') # 창 띄우기

/var/folders/vf/8b5vg3c94n1dx9pntt235qc80000gn/T/ipykernel_2321/3060802779.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/koojaehong/Jupyter_notebook/chromedriver') # 창 띄우기


In [4]:
driver.get('https://map.naver.com/') # 네이버 지도 이동

In [ ]:
##### Naver 리뷰 값 가져오기
#### 검색창 클릭 & 검색 값 입력
#### 왼편 searchIframe 전환 (여러 iframe으로 갖춰져 있다)
#### 1개 카드 클릭
#### 전체 frame으로 복귀
#### 오른편 entryIframe 전환
#### 가게 이름 & 별점 가져오기 (별점이 기준 값 미만이면 가져오지 않는다)
#### 전체 frame으로 복귀
#### -> 반복 10개까지

##### Kakao 검색
#### ...

In [3]:
### Naver 맛집 검색, 검색어(지역+메뉴), 기준 별점, 찾을 범위
### ! 속도는 본인 사양에 맞춰서 조정
def naver_finder(place, menu, standard_star, count): # , count
    search_term = place + " " + menu
    
    # 검색어 기준으로 네이버 지도 검색
    driver.get('https://map.naver.com/v5/search/{}'.format(search_term))
    
    time.sleep(3)
    try:
        # 찾은 맛집을 담을 빈 데이터 프레임
        naver_df = pd.DataFrame()
        
        # 1부터 찾을범위까지 반복하여 검색
        for num in range(1, count+1):
            # frame이 크게 3개로 나뉜다: 전체, searchIframe(왼편), entryIfrmae(우편), default_content(기본)
            
            # WebDriverWait 활용 searchIframe 대기 후 전환
            WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.ID, "searchIframe")))
            searchIframe = driver.find_element_by_id("searchIframe")
            driver.switch_to.frame(searchIframe)
            
            # 가게 하나씩 클릭 가능한가? 가능하지 않다면 스크롤 다운 후 기다렸다가 클릭
            try:
                card_click = driver.find_element_by_xpath(
                '//*[@id="_pcmap_list_scroll_container"]/ul/li[{}]/div[1]/a'.format(num))
                card_click.click()
            except:
                driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
            
            WebDriverWait(driver, 2).until(EC.element_to_be_clickable(
                    (By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li[{}]/div[1]/a'.format(num))))
            card_click = driver.find_element_by_xpath(
                '//*[@id="_pcmap_list_scroll_container"]/ul/li[{}]/div[1]/a'.format(num))
            driver.execute_script("arguments[0].click();", card_click)
#             try:
#                 WebDriverWait(driver, 2).until(EC.element_to_be_clickable(
#                     (By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li[{}]/div[1]/a'.format(num))))
#                 card_click = driver.find_element_by_xpath(
#                     '//*[@id="_pcmap_list_scroll_container"]/ul/li[{}]/div[1]/a'.format(num))
#                 card_click.click()
#             except:
#                 driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
#                 WebDriverWait(driver, 2).until(EC.element_to_be_clickable(
#                     (By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li[{}]/div[1]/a'.format(num))))
#                 card_click = driver.find_element_by_xpath(
#                     '//*[@id="_pcmap_list_scroll_container"]/ul/li[{}]/div[1]/a'.format(num))
#                 driver.execute_script("arguments[0].click();", card_click)
            
            # 기본 설정으로 되돌아오기
            time.sleep(1)
            driver.switch_to.default_content()
            
            # WebDriverWait 활용 entryIframe 대기 후 전환
            WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, "entryIframe")))
            entryIframe = driver.find_element_by_id("entryIframe")
            driver.switch_to.frame(entryIframe)
            
            # 우편의 가게 이름 변수로 지정 → restaurant_name
            WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CLASS_NAME, '_3XamX')))
            restaurant_name = driver.find_element_by_class_name('_3XamX').text
            
            # 해당 가게의 별점 변수로 지정 → review_star
            # 단, 조건이 있음. 별점이 없는 경우, 0으로 계산한다.
            # 별점을 구하기 위해서는 정규표현식으로 클렌징 진행, '\n'과 '/5'를 없애주고 float 타입으로 변경
            try:
                WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, '_1Y6hi._1A8_M')))
                review_star = driver.find_element_by_class_name('_1Y6hi._1A8_M').text
                review_star = re.sub('별점\n|/5', '', review_star)
                review_star = float(review_star)
            except:
                review_star = 0
                pass
            
            # 다음 반복을 위해 기본 설정으로 되돌아오기
            time.sleep(1)
            driver.switch_to.default_content()
            
            # 별점이 기준 별점보다 높은 경우에만 진행, 아닌 경우 그냥 다음 반복을 위해 pass
            # 가게명과 별점을 지정하여 데이터 프레임화
            # 전체 데이터 프레임과 해당 반복 구간의 데이터 프레임을 합친다(계속 반복하여 합치면 전체 데이터 프레임에 모두 들어감)
            if review_star > standard_star:
                df = pd.DataFrame({'가게명': [restaurant_name], '네이버 별점': [review_star]})
                naver_df = pd.concat([naver_df, df])
            else:
                pass
            
        # 중복값 제거
        naver_df.drop_duplicates()
    
    except:
        pass
    # 모든 예외의 에러 메시지를 출력할 때는 Exception을 사용
#     except Exception as e:
#         print('예외가 발생했습니다.', e)
#         logging.error(traceback.format_exc())
    
    return naver_df

In [4]:
def kakao_finder(place, name):
    kakao_search_term = place + " " + name
    
    try:
        WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, "search.keyword.query")))
        driver.find_element_by_id('search.keyword.query').clear()
        driver.find_element_by_id('search.keyword.query').send_keys(kakao_search_term)
        driver.find_element_by_id('search.keyword.query').send_keys(Keys.RETURN)
        
        time.sleep(2)
        kakao_star = driver.find_element_by_xpath('//*[@id="info.search.place.list"]/li[1]/div[4]/span[1]/em').text
        kakao_star = float(kakao_star)
    
    except:
        kakao_star = '해당 없음'
        pass

#         except Exception as e:
#             kakao_star = '해당 없음'
#             print('예외가 발생했습니다.', e)
#             logging.error(traceback.format_exc())
    
    return kakao_star

In [13]:
def taste_finder(place, menu, standard_star, count):
    
    # 네이버 플레이스 이동
    driver.get('https://map.naver.com/')
    
    # 네이버 검색 결과 저장
    taste_df = naver_finder(place, menu, standard_star, count)
        
    # 카카오맵 이동
    driver.get('https://map.kakao.com/')
    
    taste_df['카카오 별점'] = 0
    # 카카오 별점 반복
    try:
        for num in range(0, len(taste_df)):
            pass_place = taste_df['가게명'].iloc[num]
            kakao_star = kakao_finder(place, pass_place)
            taste_df['카카오 별점'].iloc[num] = kakao_star
    
        if len(taste_df) < 1:
            taste_df = "검색 결과, 충족하는 값이 없습니다."
        else:
            pass
    
    except:
        pass
    
    return taste_df

In [16]:
a = taste_finder('소고기', '석촌역', 4.5, 15)

/Users/koojaehong/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [17]:
a

,가게명,네이버 별점,카카오 별점
0,우마담 호수,4.79,3.3
0,패대기 서울석촌점,4.83,해당 없음
